<a href="https://colab.research.google.com/github/knightkjt/FSRCNN_yippp_pub/blob/master/fsrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import os
import sys


     |████████████████████████████████| 993kB 5.2MB/s 


In [1]:
!ls gdrive/My\ Drive\

 영어	   '쉬식시간 활용'     dataset		        IDs.gdoc        Udacity
 여행	    알고리즘공부       DeepLearning	        MentorSeminar   Video
 출장	    3DMAX_Mine	       DL_HW_ACCEL_DOC	        Music	        VQAS
 집짓기     3Dprint	       DOCs		        MyDrivers
 집정리     AndroidApp	       Fun		        MyProfile
 핸드폰     Arduino	      '내가 저장한 장소.gmap'   Program
 멀티콥터  'Colab Notebooks'  'Google Buzz'	        Python
